# 01 - Image Analysis

References:
 - https://python.langchain.com/docs/integrations/llms/ollama/
  -


In [ ]:
%pip install -q langchain langchain-ollama langchain-chroma ipython

In [ ]:
from PIL import Image
from langchain_ollama import OllamaLLM
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import chain

from util import convert_to_base64

In [ ]:
# Globals
# from langchain import globals
#globals.set_debug(True)

dataset_directory = "/mnt/network-data/storage/projects/vf/data"

vision_model = "llama3.2-vision"
ollama_api_endpoint = "http://jupiter.btech.software:11434"
llm = OllamaLLM(
    model=vision_model,
    base_url=ollama_api_endpoint,
    temperature=0.5,
    num_predict=1024
)

query = "Describe this image"

In [ ]:
image_path = f"{dataset_directory}/DataStock/NOVA_Apresentação Comercial DataStock-page-11.jpg"
image_b64 = convert_to_base64(Image.open(image_path))
response = llm.bind(images=[image_b64]).invoke(query)
print(response)

### Include Response Model

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class ImageInformation(BaseModel):
    """Information about an image."""
    image_description: str = Field(description="a short description of the image")
    image_description_pt: str = Field(description="a short description of the image in portuguese (pt_BR)")
    brands: list[str] = Field(description="list of brands on the picture")
    companies: list[str] = Field(description="list of companies on the picture")
    sales_guidance: str = Field(
        description="a business insight on how this information can support sales professionals")

parser = JsonOutputParser(pydantic_object=ImageInformation)

In [ ]:
prompt = PromptTemplate(
    template="""

    \n{format_instructions}\n{query}\n
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

vision_chain = prompt | llm.bind(images=[image_b64]) | parser
vision_chain.invoke({'query': "Analyse the image."})